\begin{align*}
u_{tt} &= u_{xx} \\
u_{tt}(t,x) &= \frac{u(t,x+ \Delta x)-2u(t,x)+u(t,x- \Delta x)}{\Delta x^{2}} \\
u_{tt}(t,x) + \frac{2}{\Delta x^{2}}u(t,x) &= \frac{u(t,x+ \Delta x)+u(t,x- \Delta x)}{\Delta x^{2}} \\
\end{align*}
Introduce: $q = \frac{\sqrt{2}}{\Delta x}$.
\begin{align*}
u_{tt}(t,x) + q^{2}u(t,x) &= q^{2}\frac{u(t,x+ \Delta x)+u(t,x- \Delta x)}{2} \\
\end{align*}
Green's functions:

\begin{align*}
u(t,x) = \cos\left(q t\right) u(0,t) + \frac{\sin\left(qt\right)}{q} u_t(0,x) +    
\frac{q}{2} \int_{0}^{t} \sin\left(q(s-t) \right) \left[ u(s,x+\Delta x) +u(s,x-\Delta x) \right] ds 
\end{align*}



In [24]:
from random import random as U
import numpy as np
from numba import njit, prange
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

# @njit(fastmath=True)
# def u(t,x): return x**2 + 2*t #needs to satisfy wave equation
# @njit(fastmath=True)
# def ut(t,x): return  2 #needs to satisfy wave equation

# @njit(fastmath=True)
# def u(t,x): return x*t**2 + x**3/3 #needs to satisfy wave equation
# @njit(fastmath=True)
# def ut(t,x): return 2*x*t  #needs to satisfy wave equation

@njit(fastmath=True)
def u(t,x): return x**2 *t**2/2 + t**4/12 + x**4/12 #needs to satisfy wave equation
@njit(fastmath=True)
def ut(t,x): return x**2*t+t**3/3  #needs to satisfy wave equation

# this looks like the most challenging solution
# @njit(fastmath=True)
# def u(t,x): return np.sin((x+t)) #needs to satisfy wave equation
# @njit(fastmath=True)
# def ut(t,x): return np.cos((x+t))  #needs to satisfy wave equation

@njit(fastmath=True)
def P1(t,q): return np.cos( q*t) 
@njit(fastmath=True)
def P2(t,q): return np.sin(q*t)/q 
@njit(fastmath=True)
def GG(t,s,q): return q*np.sin(q*(t-s)) # not sure this is good ? 
count = 0

@njit(fastmath=True)
def Z0(t,x,dx,dt):
    if abs(x)>5: return u(t,x) # with or without boundaries 
    q =np.sqrt(2)/dx
    s = t*U()
    xnew = x+dx if U()<1/2 else x-dx
    sol = P1(t,q)*u(0,x)
    sol += P2(t,q)*ut(0,x)
    if t/dt>1:
        sol+=t*GG(t,s,q)*Z0(s,xnew,dx,dt)
    elif U()<t/dt:
        sol+=dt*GG(t,s,q)*Z0(s,xnew,dx,dt)
    return sol

@njit(fastmath=True, parallel=True)
def z0(t,x,dx,dt, nsim):
    total = 0.0 # <- typing becomes important 0 vs 0.0
    total2 = 0.0 # <- typing becomes important 0 vs 0.0
    for _ in prange(nsim): # numba likes explicit loops sometimes even over vectorized numpy
        total += Z0(t,x,dx,dt)
        total2 += Z0(t,x,dx,dt)**2
    return total/nsim,total2/nsim-(total/nsim)**2

def update_histogram(t,x, dt,dx,nsim):
    nsim = 10**int(nsim)
    sol,v = z0(t,x,dx,dt,nsim)
    avg_label.value = f"Updated Average of zz: {sol:.2f}, {u(t,x):.2f}, acc: {100*(sol-u(t,x))/(u(t,x)+0.00001):.2f}%"
    var.value = f"sig: {np.sqrt(v/nsim)}"
    if nsim<=10**3:
        zz = np.array([Z0(t, x, dx,dt) for _ in range(nsim)])
        plt.hist(zz, bins=int(nsim/20+1))
        plt.xlabel("Values of Z")
        plt.ylabel("Frequency")
        plt.title("Histogram of Z")
        plt.show()

dt_slider = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.01, description='dt:')
dx_slider = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.01, description='dx:')
t_slider = widgets.FloatSlider(value=0, min=0, max=10.0, step=0.1, description='t:')
x_slider = widgets.FloatSlider(value=0, min=-50, max=50, step=0.1, description='x:')
nsim_slider = widgets.FloatSlider(value=0, min=0, max=6, step=1, description='nsim:')

avg_label = widgets.Label(value="")
var = widgets.Label(value="")

display(avg_label)
display(var)

interact(update_histogram,t=t_slider,x=x_slider, dt=dt_slider, dx=dx_slider, nsim = nsim_slider)

Label(value='')

Label(value='')

interactive(children=(FloatSlider(value=0.0, description='t:', max=10.0), FloatSlider(value=0.0, description='…

<function __main__.update_histogram(t, x, dt, dx, nsim)>

move a period and take the difference

\begin{align*}
u(t+\frac{2 \pi}{q},x)  = u(t,x) +
\frac{q}{2} \int_{t}^{t+\frac{2 \pi}{q} } \sin\left(q(s-t) \right) \left[ u(s,x+\Delta x) +u(s,x-\Delta x) \right] ds 
\end{align*}


Control variate previous example